In [ ]:
from sage.combinat.permutation import Permutation, Permutations
from sage.matrix.constructor import matrix, block_matrix
from sage.rings.integer_ring import ZZ
from sage.combinat.symmetric_group_representations import SymmetricGroupRepresentation_generic_class


def induced_representation_matrix(g, sub_rep, coset_reps):
    """
    Compute the matrix for the induced representation of g.
    
    The induced representation Ind_H^G(ρ) of a representation ρ of subgroup H
    to group G acts on the space of functions from G/H to the representation space.
    
    Parameters:
    -----------
    g : Permutation
        Element of the symmetric group S_n
    sub_rep : function
        Function that takes a permutation and returns its representation matrix
        for the subgroup representation
    coset_reps : list of Permutations
        Representatives of the left cosets G/H
        
    Returns:
    --------
    Matrix representing g in the induced representation
    
    The formula is: [Ind(g)]_{i,j} = ρ(r_i * g * r_j^{-1}) if r_i * g * r_j^{-1} ∈ H,
                                      0 otherwise
    where r_i, r_j are coset representatives and ρ is the subgroup representation.
    """
    n = len(coset_reps)
    d = sub_rep(Permutation([1])).nrows()  # dimension of sub-representation
    
    # Build block matrix
    blocks = [[None for _ in range(n)] for _ in range(n)]
    zero_block = matrix(ZZ, d, d, 0)
    
    for i, r_i in enumerate(coset_reps):
        for j, r_j in enumerate(coset_reps):
            # Compute r_i * g * r_j^{-1}
            perm = r_i * g * r_j.inverse()
            
            # Check if this permutation is in the subgroup
            # For this implementation, we'll compute the matrix
            # If it's not in H, sub_rep should handle it or we check explicitly
            try:
                blocks[i][j] = sub_rep(perm)
            except:
                blocks[i][j] = zero_block
    
    return block_matrix(blocks)


class InducedRepresentation(SymmetricGroupRepresentation_generic_class):
    """
    Class for computing induced representations from Young subgroups.
    
    Given a representation of S_k (acting on first k elements), 
    compute its induction to S_n.
    """
    
    def __init__(self, n, k, sub_representation):
        """
        Initialize induced representation from S_k to S_n.
        
        Parameters:
        -----------
        n : int
            Size of the full symmetric group
        k : int
            Size of the subgroup (k <= n)
        sub_representation : function or SymmetricGroupRepresentation
            Representation of S_k
        """
        self.n = n
        self.k = k
        self.sub_rep = sub_representation
        
        # Get coset representatives for S_n / S_k
        # S_k acts on {1, ..., k}, so cosets correspond to ways to choose
        # which k elements are acted upon
        self.coset_reps = self._compute_coset_representatives()

    def _repr_(self):
        r"""
        Return a string representation of ``self``.

        EXAMPLES::

            sage: sym_repn_3 = SymmetricGroupRepresentation([2,1])
            sage: InducedRepresentation(4,3,sym_repn_3)
            Induced representation of symmetric group of representation associated to [2,1] from S_3 to S_4
        """
        return "Induced representation of symmetric group of representation associated to partition {} from S_{} to S_{}".format(self.sub_rep._partition, self.k, self.n)

    def _compute_coset_representatives(self):
        """
        Compute representatives of left cosets S_n / S_k.
        
        We use permutations that map {1,...,k} to different k-subsets.
        """
        from sage.combinat.combination import Combinations
        
        reps = []
        for subset in Combinations(range(1, self.n + 1), self.k):
            # Create a permutation that maps {1,...,k} to subset
            perm_list = list(range(1, self.n + 1))
            
            # Put elements of subset in first k positions
            for i, elem in enumerate(subset):
                # Find where elem is and swap it to position i
                idx = perm_list.index(elem)
                perm_list[i], perm_list[idx] = perm_list[idx], perm_list[i]
            
            reps.append(Permutation(perm_list))
        
        return reps
    
    def __call__(self, g):
        """
        Compute the induced representation matrix for element g.
        
        Parameters:
        -----------
        g : Permutation or list
            Element of S_n
            
        Returns:
        --------
        Matrix in the induced representation
        """
        if not isinstance(g, Permutation):
            g = Permutation(g)
            
        n_cosets = len(self.coset_reps)
        
        if callable(self.sub_rep):
            d = self.sub_rep(Permutation(range(1, self.k + 1))).nrows()
        else:
            d = self.sub_rep(Permutation(range(1, self.k + 1))).nrows()
        
        # Build the induced representation matrix as a block matrix
        blocks = [[matrix(ZZ, d, d, 0) for _ in range(n_cosets)] 
                  for _ in range(n_cosets)]
        
        for i, r_i in enumerate(self.coset_reps):
            for j, r_j in enumerate(self.coset_reps):
                # Compute h = r_i * g * r_j^{-1}
                h = r_i * g * r_j.inverse()
                
                # Check if h stabilizes {1,...,k} (i.e., h is in S_k × S_{n-k})
                h_list = list(h)
                first_k = sorted(h_list[:self.k])
                
                if first_k == list(range(1, self.k + 1)):
                    # h acts on first k elements like an element of S_k
                    # Extract the S_k part
                    h_k = Permutation(h_list[:self.k])
                    blocks[i][j] = self._get_sub_rep_matrix(h_k)
        
        return block_matrix(blocks)
    
    def _get_sub_rep_matrix(self, perm):
        """Helper to get representation matrix from sub_rep."""
        if callable(self.sub_rep):
            return self.sub_rep(perm)
        else:
            return self.sub_rep(perm)
    
    def dimension(self):
        """Return the dimension of the induced representation."""
        from sage.combinat.combination import Combinations
        n_cosets = len(list(Combinations(self.n, self.k)))
        
        if callable(self.sub_rep):
            d_sub = self.sub_rep(Permutation(range(1, self.k + 1))).nrows()
        else:
            d_sub = self.sub_rep(Permutation(range(1, self.k + 1))).nrows()
        
        return n_cosets * d_sub


# Example usage and testing
def example_usage():
    """
    Example: Induce the trivial representation of S_2 to S_3.
    """
    # Trivial representation of S_2
    def trivial_rep_s2(perm):
        return matrix(ZZ, 1, 1, [1])
    
    # Create induced representation
    ind_rep = InducedRepresentation(3, 2, trivial_rep_s2)
    
    print("Dimension of induced representation:", ind_rep.dimension())
    print("\nCoset representatives:")
    for i, r in enumerate(ind_rep.coset_reps):
        print(f"  r_{i} = {r}")
    
    print("\nRepresentation matrices:")
    for perm in Permutations(3):
        print(f"\nπ({perm}) =")
        print(ind_rep(perm))
    
    return ind_rep


if __name__ == "__main__":
    # This would run in Sage
    print("To use: create an InducedRepresentation object with your subgroup rep")
    print("Example: ind_rep = InducedRepresentation(4, 3, my_s3_representation)")
    print("Then: ind_rep(Permutation([2,1,3,4])) gives the matrix")

To use: create an InducedRepresentation object with your subgroup rep
Example: ind_rep = InducedRepresentation(4, 3, my_s3_representation)
Then: ind_rep(Permutation([2,1,3,4])) gives the matrix


In [16]:
example_usage()

Dimension of induced representation: 3

Coset representatives:
  r_0 = [1, 2, 3]
  r_1 = [1, 3, 2]
  r_2 = [2, 3, 1]

Representation matrices:

π([1, 2, 3]) =
[1|0|0]
[-+-+-]
[0|1|0]
[-+-+-]
[0|0|1]

π([1, 3, 2]) =
[0|1|0]
[-+-+-]
[1|0|0]
[-+-+-]
[0|0|1]

π([2, 1, 3]) =
[1|0|0]
[-+-+-]
[0|0|1]
[-+-+-]
[0|1|0]

π([2, 3, 1]) =
[0|0|1]
[-+-+-]
[1|0|0]
[-+-+-]
[0|1|0]

π([3, 1, 2]) =
[0|1|0]
[-+-+-]
[0|0|1]
[-+-+-]
[1|0|0]

π([3, 2, 1]) =
[0|0|1]
[-+-+-]
[0|1|0]
[-+-+-]
[1|0|0]


<repr(<__main__.InducedRepresentation at 0x11fc2ffd0>) failed: AttributeError: 'function' object has no attribute '_partition'>

In [17]:
sym_repn_3 = SymmetricGroupRepresentation([2,1])
print(sym_repn_3(Permutation([1,2,3])))

[1 0]
[0 1]


In [18]:
sym_repn_3 = SymmetricGroupRepresentation([2,1])
ind_rep_4_3_sym21 = InducedRepresentation(4,3,sym_repn_3)
print(ind_rep_4_3_sym21)

Induced representation of symmetric group of Specht module associated to [2, 1] from S_3 to S_4


In [6]:
    # Trivial representation of S_2
    def trivial_rep_s2(perm):
        return matrix(ZZ, 1, 1, [1])
    
    # Create induced representation
    ind_rep = InducedRepresentation(3, 2, trivial_rep_s2)

In [7]:
ind_rep.dimension()

3

In [8]:
ind_rep(Permutation([1,2,3]))

[1|0|0]
[-+-+-]
[0|1|0]
[-+-+-]
[0|0|1]

In [9]:
mat_321 = ind_rep_4_3_sym21(Permutation([3,2,1])); mat_321

[-1  1| 0  0| 0  0| 0  0]
[ 0  1| 0  0| 0  0| 0  0]
[-----+-----+-----+-----]
[ 0  0| 0  0| 0  0| 0 -1]
[ 0  0| 0  0| 0  0|-1  0]
[-----+-----+-----+-----]
[ 0  0| 0  0| 0 -1| 0  0]
[ 0  0| 0  0|-1  0| 0  0]
[-----+-----+-----+-----]
[ 0  0| 0 -1| 0  0| 0  0]
[ 0  0|-1  0| 0  0| 0  0]

In [10]:
mat_321*mat_321

[1 0 0 0 0 0 0 0]
[0 1 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 0 0 0 1 0 0 0]
[0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 1]